# Projeto Final

## Primeira Entrega:
- Escolher uma API da lista disponível;
- Fazer a extração de pelo menos 3 tabelas;
- Criar um alerta de erro usando a função alerta() do desafio da aula Fundamentos de programação III.

In [4]:
import requests
import pandas as pd
from datetime import datetime
from plyer import notification

In [5]:
## Criar um alerta de erro usando a função alerta()

def alerta(nivel, base, etapa):
    now = str(datetime.now())
    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}"

    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")

    notification.notify(
        title=title,
        message=msg,
        app_name='alerta',
        timeout=10
    )


In [6]:
## Acessar API

url = 'https://restcountries.com/v3.1/all'
response = requests.get(url)

if response.status_code == 200:
    data_json = response.json()
    print(response)
else:
    alerta(nivel=3, base="REST Countries", etapa="EXTRACAO")

<Response [200]>


In [59]:
## Extrair primeira tabela
response = requests.get('https://restcountries.com/v3.1/all')
data = response.json()

countries = pd.DataFrame([{'name': country['name']['common'], 'capital': country.get('capital', [''])[0], 'region': country['region']} for country in data])

countries



,name,capital,region
0,Cyprus,Nicosia,Europe
1,Eritrea,Asmara,Africa
2,Liberia,Monrovia,Africa
3,Bermuda,Hamilton,Americas
4,Vatican City,Vatican City,Europe
...,...,...,...
245,South Sudan,Juba,Africa
246,Honduras,Tegucigalpa,Americas
247,Saint Vincent and the Grenadines,Kingstown,Americas
248,Sri Lanka,Sri Jayawardenepura Kotte,Asia


In [117]:
## Extrair segunda tabela

response = requests.get('https://restcountries.com/v3.1/all')
data = response.json()

driving = pd.DataFrame([{'country': country['name']['common'], 'side': country['car']['side'] } for country in data])

driving

,country,side
0,Cyprus,left
1,Eritrea,right
2,Liberia,right
3,Bermuda,left
4,Vatican City,right
...,...,...
245,South Sudan,right
246,Honduras,right
247,Saint Vincent and the Grenadines,left
248,Sri Lanka,left


In [9]:
## Extrair terceira tabela

response = requests.get('https://restcountries.com/v3.1/all')
data = response.json()

flags = pd.DataFrame([{'country': country['name']['common'], 'flag description': country['flags'].get('alt') } for country in data])
flags


,country,flag description
0,Cyprus,"The flag of Cyprus has a white field, at the c..."
1,Eritrea,The flag of Eritrea comprises three triangles ...
2,Liberia,The flag of Liberia is composed of eleven equa...
3,Bermuda,None
4,Vatican City,The flag of Vatican City is square shaped. It ...
...,...,...
245,South Sudan,The flag of South Sudan is composed of three e...
246,Honduras,The flag of Honduras is composed of three equa...
247,Saint Vincent and the Grenadines,The flag of Saint Vincent and the Grenadines i...
248,Sri Lanka,The flag of Sri Lanka features two large adjac...


# Tratamento das Bases
### Realizar o tratamento das bases coletadas via API do projeto final na aula 12.
- Ajuste os nomes das colunas e linhas.
- Filtros de linhas e colunas, se necessário.
- unstack e/ou stack.
- Tratamento do tipo das variáveis.
- Ajuste de missing.
- Tratamento de colunas string.
- Você pode criar novas tabelas, salvando as bases brutas e as bases tratadas.
- As bases finais precisam estar totalmente ajustadas (da melhor forma possível) e salvas no banco de dados.

In [66]:
# Ajuste nomes das colunas

df_countries = pd.read_csv('./countries.csv')
df_countries.rename(columns={'name': 'nome', 'region': 'região'}, inplace=True)

df_countries.head()


,nome,capital,região
0,Cyprus,Nicosia,Europe
1,Eritrea,Asmara,Africa
2,Liberia,Monrovia,Africa
3,Bermuda,Hamilton,Americas
4,Vatican City,Vatican City,Europe


In [17]:
# Filtrando as capitais que estão na região da Africa
df_regiao_africa = df_countries[df_countries['região'] == 'Africa'][['capital', 'região']]
df_regiao_africa.head(10)

,capital,região
1,Asmara,Africa
2,Monrovia,Africa
6,Mogadishu,Africa
7,Lusaka,Africa
13,Khartoum,Africa
15,Brazzaville,Africa
17,Nairobi,Africa
19,Yamoussoukro,Africa
20,Dakar,Africa
28,São Tomé,Africa


In [19]:
# Coloca as regiões em maiusculo
df_regiao_africa = df_countries[df_countries['região'] == 'Africa']['capital'].str.upper()
df_regiao_africa.head(10)


1           ASMARA
2         MONROVIA
6        MOGADISHU
7           LUSAKA
13        KHARTOUM
15     BRAZZAVILLE
17         NAIROBI
19    YAMOUSSOUKRO
20           DAKAR
28        SÃO TOMÉ
Name: capital, dtype: object

In [75]:
# Ajustando missing values

df_flags = pd.DataFrame(df['flags'].tolist())
df_flags['countries'] = df_countries['nome']
df_flags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   png        250 non-null    object
 1   svg        250 non-null    object
 2   alt        194 non-null    object
 3   countries  250 non-null    object
dtypes: object(4)
memory usage: 7.9+ KB


In [82]:
df_flags = df_flags.fillna({'alt':'Sem descrição'})
df_flags

,png,svg,alt,countries
0,https://flagcdn.com/w320/cy.png,https://flagcdn.com/cy.svg,"The flag of Cyprus has a white field, at the c...",Cyprus
1,https://flagcdn.com/w320/er.png,https://flagcdn.com/er.svg,The flag of Eritrea comprises three triangles ...,Eritrea
2,https://flagcdn.com/w320/lr.png,https://flagcdn.com/lr.svg,The flag of Liberia is composed of eleven equa...,Liberia
3,https://flagcdn.com/w320/bm.png,https://flagcdn.com/bm.svg,Sem descrição,Bermuda
4,https://flagcdn.com/w320/va.png,https://flagcdn.com/va.svg,The flag of Vatican City is square shaped. It ...,Vatican City
...,...,...,...,...
245,https://flagcdn.com/w320/ss.png,https://flagcdn.com/ss.svg,The flag of South Sudan is composed of three e...,South Sudan
246,https://flagcdn.com/w320/hn.png,https://flagcdn.com/hn.svg,The flag of Honduras is composed of three equa...,Honduras
247,https://flagcdn.com/w320/vc.png,https://flagcdn.com/vc.svg,The flag of Saint Vincent and the Grenadines i...,Saint Vincent and the Grenadines
248,https://flagcdn.com/w320/lk.png,https://flagcdn.com/lk.svg,The flag of Sri Lanka features two large adjac...,Sri Lanka
